## Integrantes
### Alejandro Orozco 2402036-7727
### Fernando Valencia 2401899-7727
### Carlos Botero 2400879-7727
### Brandon Rivas 2400430-7727

In [ ]:
# Instalación de librerías necesarias
!pip install -U transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

# 📂 Carga de historias clínicas desde Google Drive

Se monta Google Drive en el entorno de Colab y se accede a la carpeta que contiene los archivos de texto con historias clínicas. Cada archivo es leído línea por línea, y se almacenan únicamente las líneas no vacías en la lista `historias`, la cual contiene todas las frases que serán procesadas posteriormente por los modelos de análisis.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

carpeta_historias = "/content/drive/MyDrive/Analitica en Salud/Negación"

Mounted at /content/drive


In [ ]:
archivos = os.listdir(carpeta_historias)
print(len(archivos))

106


In [ ]:
historias = []
for archivo in archivos:
  ruta_completa = os.path.join(carpeta_historias, archivo)
  with open(ruta_completa, "r") as f:
    for linea in f:
      if linea.strip():
        historias.append(linea.strip())

In [ ]:
historias[0:10]

['PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.',
 'Antecedentes Personales:- Alergia a Fluconazol.',
 '- No HTA.',
 'No DM.',
 'No DL.',
 '- Niega habitos toxicos.- Candidiasis recurrentes- Iqx: ninguna.',
 'MEDICACIÓN- No medicación habitual.',
 'Muje de 59 años remitida desde oncología con Adenocarcinoma ductal infiltrante de mama izquierda, moderadamente diferenciado de 2 cm, intervenido mediante mastectomía radical izquierda el 20/06/1991.',
 '-Carcinoma lobulillar in situ residual de 2 cm en mama dcha, intervenida el 16/04/2003 mediante mastectomía radical derecha.',
 'Recidiva pulmonar y pleural confirmada con AP en el 2012.']

In [ ]:
from huggingface_hub import login
login()

# ❗ Carga del modelo de detección de negación e incertidumbre

Se carga el modelo `JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES` desde Hugging Face, entrenado específicamente para detectar expresiones de negación e incertidumbre en textos clínicos. Se crea un pipeline de NER con estrategia de agregación `"simple"` para identificar y agrupar tokens relacionados con estas expresiones.


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_neg = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"
tokenizer_neg = AutoTokenizer.from_pretrained(model_neg)
model_neg = AutoModelForTokenClassification.from_pretrained(model_neg)

pipeline_neg = pipeline("ner", model=model_neg, tokenizer=tokenizer_neg, aggregation_strategy="simple")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Device set to use cpu


# 🧠 Detección de negación e incertidumbre en las historias clínicas

Se aplica el modelo de negación e incertidumbre a cada línea del texto clínico. Para cada frase, se extraen los tokens etiquetados por el modelo (`contexto`) y se almacenan junto con el texto original en la lista `resultados_negacion`. Esta estructura permitirá analizar qué partes del texto están marcadas como negadas, inciertas o fuera de interés.


In [ ]:
resultados_negacion = []

for linea in historias:
    entidades_contexto = pipeline_neg(linea)
    resultados_negacion.append({
        "linea": linea,
        "contexto": entidades_contexto
    })

# 🔍 Inspección inicial de etiquetas generadas por el modelo de negación

Se imprimen los resultados de la detección de negación e incertidumbre para cada línea procesada, mostrando cada token con su respectiva etiqueta (`entity_group`). Al revisar la salida, se identificó que las etiquetas generadas por el modelo no estaban mapeadas a nombres legibles (por ejemplo, `LABEL_0`, `LABEL_1`, etc.), lo que evidenció la necesidad de realizar un análisis manual para interpretar y asignar un significado clínico a cada etiqueta.


In [ ]:
for r in resultados_negacion:
    print(f"\nTexto: {r['linea']}")
    for ent in r['contexto']:
        print(f"  → {ent['word']} ({ent['entity_group']})")



Texto: PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.
  → primera consulta de oncologia medica. (LABEL_8)

Texto: Antecedentes Personales:- Alergia a Fluconazol.
  → antecedentes personales : - alergia a fluconazol. (LABEL_8)

Texto: - No HTA.
  → - (LABEL_8)
  → no (LABEL_0)
  → h (LABEL_1)
  → ##ta (LABEL_5)
  → . (LABEL_8)

Texto: No DM.
  → no (LABEL_0)
  → dm (LABEL_1)
  → . (LABEL_8)

Texto: No DL.
  → no (LABEL_0)
  → dl (LABEL_1)
  → . (LABEL_8)

Texto: - Niega habitos toxicos.- Candidiasis recurrentes- Iqx: ninguna.
  → - (LABEL_8)
  → ni (LABEL_0)
  → ##ega habit (LABEL_1)
  → ##os toxicos (LABEL_5)
  → . - candidiasis recurrentes - iqx : (LABEL_8)
  → ning (LABEL_0)
  → ##una (LABEL_1)
  → . (LABEL_8)

Texto: MEDICACIÓN- No medicación habitual.
  → medicacion - (LABEL_8)
  → no (LABEL_0)
  → med (LABEL_1)
  → ##icacion habitual (LABEL_5)
  → . (LABEL_8)

Texto: Muje de 59 años remitida desde oncología con Adenocarcinoma ductal infiltrante de mama izquierda, moderadamente diferenciad

# 🧪 Análisis exploratorio de etiquetas para interpretación manual

Se agrupan frases completas por cada etiqueta generada por el modelo de negación (`LABEL_0`, `LABEL_1`, etc.), con el fin de facilitar su interpretación semántica. Para cada etiqueta, se recolectan y muestran al menos **dos frases de ejemplo** que contengan tokens clasificados con dicha etiqueta. Esto permite revisar en contexto cómo se utilizan las etiquetas y realizar un mapeo manual que as


In [ ]:
from collections import defaultdict

# Almacenar frases con sus tokens y etiquetas por tipo de etiqueta
frases_por_etiqueta = defaultdict(list)

# Recorremos los resultados procesados por el modelo
for resultado in resultados_negacion:
    texto = resultado["linea"]
    tokens = resultado["contexto"]

    # Obtener etiquetas presentes en esta línea
    etiquetas_en_linea = set()
    for token in tokens:
        etiqueta = token.get("entity_group") or token.get("entity") or "UNKNOWN"
        if etiqueta.startswith("LABEL_"):
            etiquetas_en_linea.add(etiqueta)

    # Añadir esta línea a cada etiqueta que esté presente
    for etiqueta in etiquetas_en_linea:
        if len(frases_por_etiqueta[etiqueta]) < 5:  # Puedes ajustar este número
            frases_por_etiqueta[etiqueta].append({
                "texto": texto,
                "tokens": [
                    {
                        "word": t["word"],
                        "etiqueta": t.get("entity_group") or t.get("entity") or "UNKNOWN"
                    }
                    for t in tokens
                ]
            })

# Mostrar al menos 2 frases por etiqueta
for etiqueta, frases in frases_por_etiqueta.items():
    print(f"\n🟩 {etiqueta} — {len(frases)} frases")
    for i, ejemplo in enumerate(frases[:2], 1):
        print(f"\n📌 Ejemplo {i}:")
        print(f"Texto: {ejemplo['texto']}")
        print("Etiquetas:")
        for token in ejemplo["tokens"]:
            print(f"  {token['word']:20} → {token['etiqueta']}")




🟩 LABEL_8 — 5 frases

📌 Ejemplo 1:
Texto: PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.
Etiquetas:
  primera consulta de oncologia medica. → LABEL_8

📌 Ejemplo 2:
Texto: Antecedentes Personales:- Alergia a Fluconazol.
Etiquetas:
  antecedentes personales : - alergia a fluconazol. → LABEL_8

🟩 LABEL_1 — 5 frases

📌 Ejemplo 1:
Texto: - No HTA.
Etiquetas:
  -                    → LABEL_8
  no                   → LABEL_0
  h                    → LABEL_1
  ##ta                 → LABEL_5
  .                    → LABEL_8

📌 Ejemplo 2:
Texto: No DM.
Etiquetas:
  no                   → LABEL_0
  dm                   → LABEL_1
  .                    → LABEL_8

🟩 LABEL_0 — 5 frases

📌 Ejemplo 1:
Texto: - No HTA.
Etiquetas:
  -                    → LABEL_8
  no                   → LABEL_0
  h                    → LABEL_1
  ##ta                 → LABEL_5
  .                    → LABEL_8

📌 Ejemplo 2:
Texto: No DM.
Etiquetas:
  no                   → LABEL_0
  dm                   → LABEL_1
  .            

## Mapeo final de etiquetas del modelo de negación e incertidumbre

El análisis de las salidas del modelo revela que la codificación sigue un esquema BIO extendido para marcar entidades **negadas** e **inciertas**, incluyendo también los *cues* (palabras disparadoras) y los *links* gramaticales que conectan negaciones con entidades.

### Mapeo propuesto

| Etiqueta   | Nombre corto sugerido | Significado (es → en)                                               | Ejemplos clave                                       |
|------------|------------------------|----------------------------------------------------------------------|------------------------------------------------------|
| **LABEL_0** | `CUE_NEG`              | Palabra o locución que expresa **negación**                         | no, ni, sin, ausencia, imposibil, niega              |
| **LABEL_1** | `B-NEG_ENT`            | **Begin** – primer token de la entidad **negada**                   | h (HTA), dm, anoma (anomalías), meta (metástasis), realiza |
| **LABEL_5** | `I-NEG_ENT`            | **Inside** – tokens subsiguientes de la entidad **negada**          | ##ta (HTA), ##os tóxicos, ##lias…, ##stasis…, ##r pericardiocentesis |
| **LABEL_4** | `LINK_NEG`             | Token puente que **vincula la negación con la entidad**             | de, ##idad de                                        |
| **LABEL_2** | `B-UNC_CUE`            | **Begin** – palabra que inicia una expresión de **incertidumbre**   | impres-, sugestiv-, probable                         |
| **LABEL_6** | `I-UNC_CUE`            | **Inside** – continuación de un disparador de incertidumbre         | ##ion diagnóstica, ##os de                          |
| **LABEL_3** | `B-UNC_ENT`            | **Begin** – primer token de una entidad **incierta**                | ex- (exantema), ma (MAV), lesi (lesión)              |
| **LABEL_7** | `I-UNC_ENT`            | **Inside** – continuación de la entidad **incierta**                | ##ante, ##ma, de probable origen medicamentoso, lesión mal definida |
| **LABEL_8** | `O`                    | **Outside** – token fuera de entidad o expresión relevante          | signos de puntuación, encabezados, conectores, texto neutro |

---


In [ ]:
label2nombre = {
    "LABEL_0": "CUE_NEG",              # Palabra que expresa negación
    "LABEL_1": "B-NEG_ENT",            # Inicio de la entidad negada
    "LABEL_2": "B-UNC_CUE",            # Inicio de cue de incertidumbre
    "LABEL_3": "B-UNC_ENT",            # Inicio de entidad incierta
    "LABEL_4": "LINK_NEG",             # Vínculo gramatical entre negación y entidad
    "LABEL_5": "I-NEG_ENT",            # Continuación de la entidad negada
    "LABEL_6": "I-UNC_CUE",            # Continuación del cue de incertidumbre
    "LABEL_7": "I-UNC_ENT",            # Continuación de la entidad incierta
    "LABEL_8": "O"                     # Fuera de interés (Outside)
}


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Cargar modelo
modelo_neg = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"
tokenizer = AutoTokenizer.from_pretrained(modelo_neg)
model = AutoModelForTokenClassification.from_pretrained(modelo_neg)

pipeline_neg = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Device set to use cpu


# ✅ Asignación de etiquetas legibles a los tokens del modelo

En este paso, se procesan nuevamente las historias clínicas con el modelo de detección de negación e incertidumbre, pero ahora se utiliza un diccionario `label2nombre` que contiene el mapeo manual previamente definido. Este diccionario traduce cada etiqueta cruda (`LABEL_0`, `LABEL_1`, etc.) a una etiqueta legible y clínicamente interpretada (como `CUE_NEG`, `B-NEG_ENT`, etc.). El resultado es una lista estructurada (`historias_procesadas`) que contiene, para cada línea, los tokens con su etiqueta original y su versión interpretada. A partir de aquí, el análisis puede centrarse en el significado semántico real de cada entidad reconocida.


In [ ]:
historias_procesadas = []

for linea in historias:
    entidades = pipeline_neg(linea)
    entidades_legibles = []

    for e in entidades:
        etiqueta_cruda = e.get("entity_group") or e.get("entity")
        nombre_etiqueta = label2nombre.get(etiqueta_cruda, "Etiqueta desconocida")

        entidades_legibles.append({
            "texto": e["word"],
            "etiqueta_modelo": etiqueta_cruda,
            "etiqueta_legible": nombre_etiqueta
        })

    historias_procesadas.append({
        "texto": linea,
        "entidades": entidades_legibles
    })


In [ ]:
for resultado in historias_procesadas:
    print(f"\n📝 Texto: {resultado['texto']}")
    for ent in resultado["entidades"]:
        print(f" → {ent['texto']} → {ent['etiqueta_legible']} ({ent['etiqueta_modelo']})")



📝 Texto: PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.
 → primera consulta de oncologia medica. → O (LABEL_8)

📝 Texto: Antecedentes Personales:- Alergia a Fluconazol.
 → antecedentes personales : - alergia a fluconazol. → O (LABEL_8)

📝 Texto: - No HTA.
 → - → O (LABEL_8)
 → no → CUE_NEG (LABEL_0)
 → h → B-NEG_ENT (LABEL_1)
 → ##ta → I-NEG_ENT (LABEL_5)
 → . → O (LABEL_8)

📝 Texto: No DM.
 → no → CUE_NEG (LABEL_0)
 → dm → B-NEG_ENT (LABEL_1)
 → . → O (LABEL_8)

📝 Texto: No DL.
 → no → CUE_NEG (LABEL_0)
 → dl → B-NEG_ENT (LABEL_1)
 → . → O (LABEL_8)

📝 Texto: - Niega habitos toxicos.- Candidiasis recurrentes- Iqx: ninguna.
 → - → O (LABEL_8)
 → ni → CUE_NEG (LABEL_0)
 → ##ega habit → B-NEG_ENT (LABEL_1)
 → ##os toxicos → I-NEG_ENT (LABEL_5)
 → . - candidiasis recurrentes - iqx : → O (LABEL_8)
 → ning → CUE_NEG (LABEL_0)
 → ##una → B-NEG_ENT (LABEL_1)
 → . → O (LABEL_8)

📝 Texto: MEDICACIÓN- No medicación habitual.
 → medicacion - → O (LABEL_8)
 → no → CUE_NEG (LABEL_0)
 → med → B-NEG_ENT (LAB